In [2]:
import funcs.data_wrangling as dw
import pandas as pd
import seaborn
import matplotlib.pyplot as plt
import plotly.express as px
from matplotlib import rcParams

rcParams['figure.figsize'] = 15, 5

import warnings
warnings.filterwarnings('ignore')

In [7]:
data = dw.ons_data(freq='h', ano_inicio=2000, ano_fim=2023, idreg="S")
df = dw.pipeline(data, update=True)

In [ ]:
df.head()

,id_reg,desc_reg,load_mwmed
date,,,
2000-01-01,S,SUL,4800.650000
2000-01-02,S,SUL,4899.800000
2000-01-03,S,SUL,6261.554167
2000-01-04,S,SUL,6733.741667
2000-01-05,S,SUL,6961.170833


In [ ]:
df.tail()

,id_reg,desc_reg,load_mwmed
date,,,
2023-02-26,S,SUL,9914.628292
2023-02-27,S,SUL,12771.632083
2023-02-28,S,SUL,13330.273542
2023-03-01,S,SUL,13529.272583
2023-03-02,S,SUL,14316.978167


In [ ]:
df.sample(5)

,id_reg,desc_reg,load_mwmed
date,,,
2023-02-22,S,SUL,12580.842458
2018-07-08,S,SUL,8480.151167
2000-07-13,S,SUL,6986.525000
2006-03-02,S,SUL,8947.529167
2011-10-29,S,SUL,8609.855000
